# Experimental notebook for Goline AI LLM test

## Config

In [ ]:
class Config:
    MODEL_NAME = "gemini-2.5-flash" 
    TEMPERATURE = 1
    TOP_P = 0.8
    TOP_K = 100

## Tools

In [ ]:
import os
import pandas as pd
from typing import Type, Optional, List
from pydantic import BaseModel, Field
from vnstock import Company
from vnstock import Quote
import pandas_ta_classic as ta
from langchain.tools import BaseTool

#============Shareholders============
class ViewShareholdersInput(BaseModel):
    """Input for viewing shareholders of a Vietnamese company."""
    symbol: str = Field(description="Stock symbol of the Vietnamese company (e.g., 'VIC', 'VCB')")


class ViewShareholdersTool(BaseTool):
    """Tool to view shareholders information of Vietnamese companies using vnstock."""
    
    name: str = "view_shareholders"
    description: str = "Get shareholders information for a Vietnamese stock symbol. Use this when you need to find information about major shareholders, ownership structure of a Vietnamese company."
    args_schema: Type[BaseModel] = ViewShareholdersInput
    return_direct: bool = False
    
    def _run(self, symbol: str) -> str:
        """Execute the tool to get shareholders information."""
        try:
            company = Company(symbol=symbol, source='TCBS')
            shareholders_data = company.shareholders()
            
            if shareholders_data is None or shareholders_data.empty:
                return f"Không tìm thấy thông tin cổ đông cho mã {symbol}"
            
            return shareholders_data.to_json(orient='records', force_ascii=False)
            
        except Exception as e:
            return f"Lỗi khi lấy thông tin cổ đông cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str) -> str:
        """Async version of the tool."""
        return self._run(symbol)

# Create an instance of the tool and invoke it
# result = ViewShareholdersTool().invoke({'symbol': 'VCB'})
# print(result)

#============Management============
class ViewManagementInput(BaseModel):
    """Input for viewing management of a Vietnamese company."""
    symbol: str = Field(description="Stock symbol of the Vietnamese company (e.g., 'VIC', 'VCB')")


class ViewManagementTool(BaseTool):
    """Tool to view management information of Vietnamese companies using vnstock."""
    
    name: str = "view_management"
    description: str = "Get management information for a Vietnamese stock symbol. Use this when you need to find information about company officers, management team, executives of a Vietnamese company."
    args_schema: Type[BaseModel] = ViewManagementInput
    return_direct: bool = False
    
    def _run(self, symbol: str) -> str:
        """Execute the tool to get management information."""
        try:
            company = Company(symbol=symbol, source='TCBS')
            management_data = company.officers(filter_by='working')
            
            if management_data is None or management_data.empty:
                return f"Không tìm thấy thông tin ban lãnh đạo cho mã {symbol}"
            
            return management_data.to_json(orient='records', force_ascii=False)
            
        except Exception as e:
            return f"Lỗi khi lấy thông tin ban lãnh đạo cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str) -> str:
        """Async version of the tool."""
        return self._run(symbol)

# Create an instance of the tool and invoke it
# result = ViewManagementTool().invoke({'symbol': 'TCB'})
# print(result)

# ============Subsidaries============
class ViewSubsidiariesInput(BaseModel):
    """Input for viewing subsidiaries of a Vietnamese company."""
    symbol: str = Field(description="Stock symbol of the Vietnamese company (e.g., 'VIC', 'VCB')")


class ViewSubsidiariesTool(BaseTool):
    """Tool to view subsidiaries information of Vietnamese companies using vnstock."""
    
    name: str = "view_subsidiaries"
    description: str = "Get subsidiaries information for a Vietnamese stock symbol. Use this when you need to find information about subsidiary companies, affiliated companies of a Vietnamese company."
    args_schema: Type[BaseModel] = ViewSubsidiariesInput
    return_direct: bool = False
    
    def _run(self, symbol: str) -> str:
        """Execute the tool to get subsidiaries information."""
        try:
            company = Company(symbol=symbol, source='TCBS')
            subsidiaries_data = company.subsidiaries()
            
            if subsidiaries_data is None or subsidiaries_data.empty:
                return f"Không tìm thấy thông tin công ty con cho mã {symbol}"
            
            return subsidiaries_data.to_json(orient='records', force_ascii=False)
            
        except Exception as e:
            return f"Lỗi khi lấy thông tin công ty con cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str) -> str:
        """Async version of the tool."""
        return self._run(symbol)

# Create an instance of the tool and invoke it
# result = ViewSubsidiariesTool().invoke({'symbol': 'TCB'})
# print(result)

# ============OHLCV============
class ViewOHLCVInput(BaseModel):
    """Input for viewing OHLCV data of a Vietnamese company."""
    symbol: str = Field(description="Stock symbol of the Vietnamese company (e.g., 'VIC', 'VCB')")
    start: str = Field(description="Start date for OHLCV data in YYYY-mm-dd format", default="2024-01-01")
    end: str = Field(description="End date for OHLCV data in YYYY-mm-dd format", default="2024-12-31")
    interval: str = Field(description="Timeframe for OHLCV data. Available options: '1m' (1 minute), '5m' (5 minutes), '15m' (15 minutes), '30m' (30 minutes), '1H' (1 hour), '1D' (1 day), '1W' (1 week), '1M' (1 month)", default="1D")
    columns: Optional[List[str]] = Field(description="List of columns to return. Available options: ['time', 'open', 'high', 'low', 'close', 'volume']. If not specified, all columns will be returned.", default=None)


class ViewOHLCVTool(BaseTool):
    """Tool to view OHLCV (Open, High, Low, Close, Volume) data of Vietnamese companies using vnstock."""
    
    name: str = "view_ohlcv"
    description: str = "Get OHLCV (Open, High, Low, Close, Volume) data for a Vietnamese stock symbol with specified timeframe and date range. You can select specific columns like 'open', 'close', 'volume', etc. Use this when you need historical price data, trading volume, or technical analysis data."
    args_schema: Type[BaseModel] = ViewOHLCVInput
    return_direct: bool = False
    
    def _run(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D", columns: Optional[list] = None) -> str:
        """Execute the tool to get OHLCV data."""
        try:
            # Validate interval
            valid_intervals = ['1m', '5m', '15m', '30m', '1H', '1D', '1W', '1M']
            if interval not in valid_intervals:
                return f"Khung thời gian không hợp lệ. Các khung thời gian có sẵn: {', '.join(valid_intervals)}"
            
            # Validate columns if provided
            available_columns = ['time', 'open', 'high', 'low', 'close', 'volume']
            if columns:
                invalid_columns = [col for col in columns if col not in available_columns]
                if invalid_columns:
                    return f"Cột không hợp lệ: {', '.join(invalid_columns)}. Các cột có sẵn: {', '.join(available_columns)}"
            
            quote = Quote(symbol=symbol, source='TCBS')
            ohlcv_data = quote.history(start=start, end=end, interval=interval)
            
            if ohlcv_data is None or ohlcv_data.empty:
                return f"Không tìm thấy dữ liệu OHLCV cho mã {symbol} từ {start} đến {end} với khung thời gian {interval}"
            
            # Filter columns if specified
            if columns:
                # Ensure 'time' is always included if other columns are selected
                if 'time' not in columns and len(columns) > 0:
                    columns = ['time'] + columns
                
                # Filter the dataframe to only include requested columns
                available_cols_in_data = [col for col in columns if col in ohlcv_data.columns]
                if available_cols_in_data:
                    ohlcv_data = ohlcv_data[available_cols_in_data]
                else:
                    return f"Không tìm thấy cột nào trong dữ liệu cho mã {symbol}"
            
            return ohlcv_data.to_json(orient='records', force_ascii=False)
            
        except Exception as e:
            return f"Lỗi khi lấy dữ liệu OHLCV cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str, start: str = "2025-10-01", end: str = "2025-11-13", interval: str = "1D", columns: Optional[list] = None) -> str:
        """Async version of the tool."""
        return self._run(symbol, start, end, interval, columns)


# result = ViewOHLCVTool().invoke({"symbol": "VCB", "start": "2025-10-01", "end": "2025-11-13", "interval": "15m", "columns": ["open", "close", "volume"]})
# print(result)

#============Volume Profile============
class CalculateTotalVolumeInput(BaseModel):
    """Input schema for Calculate Total Volume tool."""
    symbol: str = Field(description="Mã cổ phiếu (ví dụ: VCB, VIC, FPT)")
    start: str = Field(default="2024-01-01", description="Ngày bắt đầu (YYYY-MM-DD)")
    end: str = Field(default="2024-12-31", description="Ngày kết thúc (YYYY-MM-DD)")
    interval: str = Field(default="1D", description="Khung thời gian (1m, 5m, 15m, 30m, 1H, 1D, 1W, 1M)")

class CalculateTotalVolumeTool(BaseTool):
    """Tool to calculate total volume for Vietnamese stocks in a specified timeframe."""
    
    name: str = "calculate_total_volume"
    description: str = "Tính tổng khối lượng giao dịch của cổ phiếu Việt Nam trong khoảng thời gian chỉ định. Trả về tổng khối lượng."
    args_schema: Type[BaseModel] = CalculateTotalVolumeInput
    return_direct: bool = False
    
    def _run(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D") -> str:
        """Execute the tool to calculate total volume."""
        try:
            # Validate interval
            valid_intervals = ['1m', '5m', '15m', '30m', '1H', '1D', '1W', '1M']
            if interval not in valid_intervals:
                return f"Khung thời gian không hợp lệ. Các khung thời gian có sẵn: {', '.join(valid_intervals)}"
            
            quote = Quote(symbol=symbol, source='TCBS')
            ohlcv_data = quote.history(start=start, end=end, interval=interval)
            
            if ohlcv_data is None or ohlcv_data.empty:
                return f"Không tìm thấy dữ liệu OHLCV cho mã {symbol} từ {start} đến {end} với khung thời gian {interval}"
            
            # Check if volume column exists
            if 'volume' not in ohlcv_data.columns:
                return f"Không tìm thấy cột 'volume' trong dữ liệu cho mã {symbol}"
            
            # Calculate total volume
            total_volume = ohlcv_data['volume'].sum()
            
            return str(int(total_volume))
            
        except Exception as e:
            return f"Lỗi khi tính tổng khối lượng cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D") -> str:
        """Async version of the tool."""
        return self._run(symbol, start, end, interval)

# result = CalculateTotalVolumeTool().invoke({"symbol": "VCB", "start": "2024-10-01", "end": "2024-11-13", "interval": "1D"})
# print(result)


#============Technical Indicators============
class CalculateSMAInput(BaseModel):
    """Input schema for Calculate SMA tool."""
    symbol: str = Field(description="Mã cổ phiếu (ví dụ: VCB, VIC, FPT)")
    start: str = Field(default="2024-01-01", description="Ngày bắt đầu (YYYY-MM-DD)")
    end: str = Field(default="2024-12-31", description="Ngày kết thúc (YYYY-MM-DD)")
    interval: str = Field(default="1D", description="Khung thời gian (1m, 5m, 15m, 30m, 1H, 1D, 1W, 1M)")
    period: int = Field(default=20, description="Chu kỳ tính SMA (ví dụ: 20 cho SMA 20 ngày)")

class CalculateSMATool(BaseTool):
    """Tool to calculate Simple Moving Average (SMA) for Vietnamese stocks."""
    
    name: str = "calculate_sma"
    description: str = "Tính toán đường trung bình động đơn giản (SMA) cho cổ phiếu Việt Nam. Trả về dữ liệu OHLCV kèm theo cột SMA."
    args_schema: Type[BaseModel] = CalculateSMAInput
    return_direct: bool = False
    
    def _run(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D", period: int = 20) -> str:
        """Execute the tool to calculate SMA."""
        try:
            # Validate interval
            valid_intervals = ['1m', '5m', '15m', '30m', '1H', '1D', '1W', '1M']
            if interval not in valid_intervals:
                return f"Khung thời gian không hợp lệ. Các khung thời gian có sẵn: {', '.join(valid_intervals)}"
            
            # Validate period
            if period <= 0:
                return f"Chu kỳ SMA phải là số dương. Giá trị nhận được: {period}"
            
            quote = Quote(symbol=symbol, source='TCBS')
            ohlcv_data = quote.history(start=start, end=end, interval=interval)
            
            if ohlcv_data is None or ohlcv_data.empty:
                return f"Không tìm thấy dữ liệu OHLCV cho mã {symbol} từ {start} đến {end} với khung thời gian {interval}"
            
            # Calculate SMA using pandas_ta
            if 'close' not in ohlcv_data.columns:
                return f"Không tìm thấy cột 'close' trong dữ liệu cho mã {symbol}"
            
            # Calculate Simple Moving Average using pandas_ta
            ohlcv_data[f'SMA_{period}'] = ta.sma(ohlcv_data['close'], length=period)
            
            # Round SMA values to 2 decimal places for better readability
            ohlcv_data[f'SMA_{period}'] = ohlcv_data[f'SMA_{period}'].round(2)
            
            return ohlcv_data.to_json(orient='records', force_ascii=False)
            
        except Exception as e:
            return f"Lỗi khi tính toán SMA cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D", period: int = 20) -> str:
        """Async version of the tool."""
        return self._run(symbol, start, end, interval, period)

# result = CalculateSMATool().invoke({"symbol": "VCB", "start": "2024-10-01", "end": "2024-11-13", "interval": "1D", "period": 20})
# print(result)

#============RSI============
class CalculateRSIInput(BaseModel):
    """Input schema for Calculate RSI tool."""
    symbol: str = Field(description="Mã cổ phiếu (ví dụ: VCB, VIC, FPT)")
    start: str = Field(default="2024-01-01", description="Ngày bắt đầu (YYYY-MM-DD)")
    end: str = Field(default="2024-12-31", description="Ngày kết thúc (YYYY-MM-DD)")
    interval: str = Field(default="1D", description="Khung thời gian (1m, 5m, 15m, 30m, 1H, 1D, 1W, 1M)")
    period: int = Field(default=14, description="Chu kỳ tính RSI (ví dụ: 14 cho RSI 14 ngày)")

class CalculateRSITool(BaseTool):
    """Tool to calculate Relative Strength Index (RSI) for Vietnamese stocks."""
    
    name: str = "calculate_rsi"
    description: str = "Tính toán chỉ số sức mạnh tương đối (RSI) cho cổ phiếu Việt Nam. Trả về dữ liệu OHLCV kèm theo cột RSI."
    args_schema: Type[BaseModel] = CalculateRSIInput
    return_direct: bool = False
    
    def _run(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D", period: int = 14) -> str:
        """Execute the tool to calculate RSI."""
        try:
            # Validate interval
            valid_intervals = ['1m', '5m', '15m', '30m', '1H', '1D', '1W', '1M']
            if interval not in valid_intervals:
                return f"Khung thời gian không hợp lệ. Các khung thời gian có sẵn: {', '.join(valid_intervals)}"
            
            # Validate period
            if period <= 0:
                return f"Chu kỳ RSI phải là số dương. Giá trị nhận được: {period}"
            
            quote = Quote(symbol=symbol, source='TCBS')
            ohlcv_data = quote.history(start=start, end=end, interval=interval)
            
            if ohlcv_data is None or ohlcv_data.empty:
                return f"Không tìm thấy dữ liệu OHLCV cho mã {symbol} từ {start} đến {end} với khung thời gian {interval}"
            
            # Calculate RSI using pandas_ta
            if 'close' not in ohlcv_data.columns:
                return f"Không tìm thấy cột 'close' trong dữ liệu cho mã {symbol}"
            
            # Calculate RSI using pandas_ta
            ohlcv_data[f'RSI_{period}'] = ta.rsi(ohlcv_data['close'], length=period)
            
            # Round RSI values to 2 decimal places for better readability
            ohlcv_data[f'RSI_{period}'] = ohlcv_data[f'RSI_{period}'].round(2)
            
            return ohlcv_data.to_json(orient='records', force_ascii=False)
            
        except Exception as e:
            return f"Lỗi khi tính toán RSI cho mã {symbol}: {str(e)}"
    
    async def _arun(self, symbol: str, start: str = "2024-01-01", end: str = "2024-12-31", interval: str = "1D", period: int = 14) -> str:
        """Async version of the tool."""
        return self._run(symbol, start, end, interval, period)

# result = CalculateRSITool().invoke({"symbol": "VCB", "start": "2024-10-01", "end": "2024-11-13", "interval": "1D", "period": 28})
# print(result)

#============Tool Calling ============
all_tools = [
    ViewOHLCVTool(),
    ViewManagementTool(),
    ViewShareholdersTool(),
    ViewSubsidiariesTool(),
    CalculateTotalVolumeTool(),
    CalculateSMATool(),
    CalculateRSITool()
]


## Chain

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
import datetime

## Chain
today = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=7))).strftime("%Y-%m-%d")

# --- 1. Load LLM and Tools ---
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = google_api_key

llm = ChatGoogleGenerativeAI(
    model=Config.MODEL_NAME,
    temperature=Config.TEMPERATURE,
    top_k=Config.TOP_K,
    top_p=Config.TOP_P
)

# --- 3. Initialize Tools with Middleware ---
original_tools = [
    ViewOHLCVTool(),
    ViewManagementTool(),
    ViewShareholdersTool(),
    ViewSubsidiariesTool(),
    CalculateTotalVolumeTool(),
    CalculateSMATool(),
    CalculateRSITool()
]

direct_tools = [
    ViewOHLCVTool(return_direct=True),
    ViewManagementTool(return_direct=True),
    ViewShareholdersTool(return_direct=True),
    ViewSubsidiariesTool(return_direct=True),
    CalculateTotalVolumeTool(return_direct=True),
    CalculateSMATool(return_direct=True),
    CalculateRSITool(return_direct=True)
]

# =============Classifer train=============
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

class QueryClassification(BaseModel):
    is_complex: bool = Field(description="True nếu câu hỏi phức tạp, False nếu đơn giản")
    reasoning: str = Field(description="Lý do phân loại")

class ComplexQuery(BaseModel):
    query: str = Field(description="Câu hỏi phức tạp cần phân tích và trả lời")

# Create classifier chain
def create_classifier_chain(llm):
    """Tạo chain để phân loại câu hỏi."""
    
    # Parser để chuyển đổi output thành Pydantic model
    parser = PydanticOutputParser(pydantic_object=QueryClassification)
    
    # Prompt template cho classifier
    classifier_prompt = ChatPromptTemplate.from_messages([
        ("system", """Bạn là một chuyên gia phân loại câu hỏi tài chính.
        Nhiệm vụ: Phân loại câu hỏi thành True (phức tạp) hoặc False (đơn giản).
        
        Câu hỏi ĐƠN GIẢN (False): Chỉ cần 1 công cụ để trả lời (ví dụ: giá cổ phiếu, thông tin cơ bản)
        Câu hỏi PHỨC TẠP (True): Cần nhiều công cụ hoặc phân tích sâu (ví dụ: so sánh, xu hướng, dự đoán)
        
        {format_instructions}"""),
        ("human", "Phân loại câu hỏi sau: {query}")
    ])
    
    # Format instructions cho parser
    classifier_prompt = classifier_prompt.partial(
        format_instructions=parser.get_format_instructions()
    )
    
    # Tạo chain
    classifier_chain = classifier_prompt | llm | parser
    
    return classifier_chain

# =============Decompose complex query=============
class QueryDecomposition(BaseModel):
    sub_queries: list[str] = Field(description="Danh sách các câu hỏi con được phân tách từ câu hỏi phức tạp")
    reasoning: str = Field(description="Lý do phân tách câu hỏi")

def create_decomposition_chain(llm):
    """Tạo chain để phân tách câu hỏi phức tạp thành các câu hỏi con."""
    
    # Parser để chuyển đổi output thành Pydantic model
    parser = PydanticOutputParser(pydantic_object=QueryDecomposition)
    
    # Prompt template cho decomposition
    decomposition_prompt = ChatPromptTemplate.from_messages([
        ("system", """Bạn là một chuyên gia phân tích câu hỏi tài chính.
        Nhiệm vụ: Phân tách câu hỏi phức tạp thành các câu hỏi con đơn giản hơn.
        
        Nguyên tắc phân tách:
        - Mỗi câu hỏi con chỉ cần 1 công cụ để trả lời
        - Các câu hỏi con phải có thứ tự logic
        - Đảm bảo câu hỏi con bao phủ toàn bộ câu hỏi gốc
        - Sử dụng mã cổ phiếu cụ thể nếu có
        
        Ví dụ:
        Câu hỏi: "So sánh hiệu suất VIC và VHM trong 3 tháng qua"
        Phân tách thành:
        1. "Xem dữ liệu OHLCV của VIC trong 3 tháng qua"
        2. "Xem dữ liệu OHLCV của VHM trong 3 tháng qua"
        3. "Tính RSI của VIC"
        4. "Tính RSI của VHM"
        
        {format_instructions}"""),
        ("human", "Phân tách câu hỏi sau: {query}")
    ])
    
    # Format instructions cho parser
    decomposition_prompt = decomposition_prompt.partial(
        format_instructions=parser.get_format_instructions()
    )
    
    # Tạo chain
    decomposition_chain = decomposition_prompt | llm | parser
    
    return decomposition_chain

def decompose_complex_query(query, llm):
    """Phân tách câu hỏi phức tạp thành các câu hỏi con."""
    decomposition_chain = create_decomposition_chain(llm)
    result = decomposition_chain.invoke({"query": query})
    return result.sub_queries

#==========Combine result==========
class CombinedResult(BaseModel):
    combined_answer: str = Field(description="Câu trả lời tổng hợp từ nhiều kết quả con")
    summary: str = Field(description="Tóm tắt ngắn gọn")

def create_combine_chain(llm):
    """Tạo chain để kết hợp nhiều kết quả thành một câu trả lời tổng hợp."""
    
    # Parser để chuyển đổi output thành Pydantic model
    parser = PydanticOutputParser(pydantic_object=CombinedResult)
    
    # Prompt template cho combining results
    combine_prompt = ChatPromptTemplate.from_messages([
        ("system", """Bạn là một chuyên gia phân tích tài chính.
        Nhiệm vụ: Kết hợp nhiều kết quả từ các câu hỏi con thành một câu trả lời tổng hợp, mạch lạc.
        
        Nguyên tắc kết hợp:
        - Tổng hợp thông tin từ tất cả các kết quả
        - Tạo câu trả lời mạch lạc, dễ hiểu
        - So sánh và phân tích nếu có nhiều mã cổ phiếu
        - Đưa ra kết luận và khuyến nghị nếu phù hợp
        - Sử dụng tiếng Việt và định dạng dễ đọc
        
        {format_instructions}"""),
        ("human", """Câu hỏi gốc: {original_query}
        
        Các kết quả từ câu hỏi con:
        {results}
        
        Hãy kết hợp các kết quả trên thành một câu trả lời tổng hợp.""")
    ])
    
    # Format instructions cho parser
    combine_prompt = combine_prompt.partial(
        format_instructions=parser.get_format_instructions()
    )
    
    # Tạo chain
    combine_chain = combine_prompt | llm | parser
    
    return combine_chain

def combine_results(original_query, results, llm):
    """Kết hợp nhiều kết quả thành một câu trả lời tổng hợp."""
    combine_chain = create_combine_chain(llm)
    
    # Format results thành string
    formatted_results = ""
    for i, result in enumerate(results, 1):
        formatted_results += f"Kết quả {i}:\n{result.get('output', str(result))}\n\n"
    
    combined_result = combine_chain.invoke({
        "original_query": original_query,
        "results": formatted_results
    })
    
    return combined_result


# =============Create tool calling agent=============
# Create system prompt for the agent
system_prompt = """Bạn là một trợ lý tài chính thông minh chuyên về thị trường chứng khoán Việt Nam.

NHIỆM VỤ CHÍNH:
- Trả lời các câu hỏi về cổ phiếu, công ty niêm yết trên sàn chứng khoán Việt Nam
- Phân tích dữ liệu tài chính và đưa ra nhận định khách quan
- Cung cấp thông tin chính xác dựa trên dữ liệu từ vnstock

NGUYÊN TẮC LÀM VIỆC:
1. Luôn sử dụng các công cụ có sẵn để truy xuất dữ liệu thực tế
2. Trả lời bằng tiếng Việt, ngôn ngữ rõ ràng, dễ hiểu
3. Định dạng kết quả dưới dạng bảng hoặc danh sách khi phù hợp
4. Nếu không tìm thấy thông tin, hãy nói rõ và đề xuất cách khác
5. Đưa ra phân tích khách quan, không khuyến nghị mua/bán cụ thể

CÁCH XỬ LÝ CÂU HỎI:
- Xác định mã cổ phiếu từ câu hỏi (VD: VIC, VCB, FPT...)
- Chọn công cụ phù hợp để lấy dữ liệu
- Phân tích và trình bày kết quả một cách có hệ thống
- Giải thích các chỉ số kỹ thuật nếu được hỏi

LƯU Ý:
- Hôm nay là ngày {today}
- Dữ liệu có thể có độ trễ, hãy thông báo nếu cần thiết
- Luôn kiểm tra tính hợp lệ của mã cổ phiếu trước khi truy xuất dữ liệu"""

# Get today's date for the prompt
today = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=7))).strftime("%Y-%m-%d")
system_prompt = system_prompt.format(today=today)

agent = create_agent(model=llm, 
                     tools=all_tools, 
                     system_prompt=system_prompt,
                     debug=False)

agent_direct = create_agent(model=llm, 
                            tools=direct_tools, 
                            system_prompt=system_prompt,
                            debug=False)

# Create chain
classifier_chain = create_classifier_chain(llm)
while True:
    query = input("Nhập câu hỏi: ")
    if query == "exit":
        break
    result = classifier_chain.invoke({"query": query})
    print(result.is_complex)
    
    if result.is_complex:
        # Decompose complex query
        decomposed_queries = decompose_complex_query(query, llm)
        
        # Create agent for each decomposed query and get direct tool outputs
        results = []
        for sub_query in decomposed_queries:
            agent_result = agent_direct.invoke({"messages": [{"role": "user", "content": sub_query}]})
            results.append(agent_result)
        
        combined_result = combine_results(query, results, llm)
        # print all tool responses
        print(combined_result)
        
    else:
        result = agent.invoke({"messages": [{"role": "user", "content": query}]})
        # print AI response
        print(result['messages'][-1].content)


True
combined_answer='Để xác định mã cổ phiếu có giá mở cửa thấp nhất trong 10 ngày qua giữa BID, TCB và VCB, chúng ta cần phân tích dữ liệu giá mở cửa (open price) của từng mã trong khoảng thời gian này.\n\n*   **Đối với BID:** Giá mở cửa thấp nhất trong 10 ngày qua là **37.1**.\n*   **Đối với TCB:** Giá mở cửa thấp nhất trong 10 ngày qua là **33.3**.\n*   **Đối với VCB:** Giá mở cửa thấp nhất trong 10 ngày qua là **59.1**.\n\nSo sánh các mức giá mở cửa thấp nhất này, mã **TCB** có giá mở cửa thấp nhất là 33.3 trong 10 ngày qua.' summary='Mã cổ phiếu TCB có giá mở cửa thấp nhất trong 10 ngày qua với mức giá 33.3.'
